In [1]:
import numpy as np
import pandas as pd
import json
from importlib import reload 
import statsmodels.api as sm
import bright_future_base as bf
reload(bf)

<module 'bright_future_base' from '\\\\wsl$\\Ubuntu-20.04\\home\\syoro\\bright-future\\src\\bright_future_base.py'>

# Example 1 with BrightFuture

In [2]:
df_works_author = bf.load_df("works", grouped_by="author")
df_awards_author = bf.load_df("awards", grouped_by="author")
df_merged = bf.merge_datasets(["works", "awards"])

best_model = bf.model_builder(data=df_merged, 
                responder="award_count",
                 predictors=["adjustedcount", "count"],
                 display="all")

Iteration's best model Features: ['count']
Iteration's best model r2: 0.05195286058228554
Iteration's best model Features: ['count', 'adjustedcount']
Iteration's best model r2: 0.05263145367997679
Overall Best model Features: ['count']
Overall Best model summary:
                            OLS Regression Results                            
Dep. Variable:            award_count   R-squared:                       0.052
Model:                            OLS   Adj. R-squared:                  0.052
Method:                 Least Squares   F-statistic:                     790.3
Date:                Fri, 13 May 2022   Prob (F-statistic):          2.49e-169
Time:                        01:23:30   Log-Likelihood:                -6008.1
No. Observations:               14423   AIC:                         1.202e+04
Df Residuals:                   14421   BIC:                         1.204e+04
Df Model:                           1                                         
Covariance Type:         

# Example 1 without BrightFuture

In [3]:
df = pd.read_csv('../data/awards-data.tsv', sep='\t', header=0)

with open('../data/csrankings/authors-small.json') as f:
    authors_small = json.load(f)
df_authors = pd.DataFrame.from_dict(authors_small)
df_authors['count'] = df_authors['count'].astype(float)
df_authors['adjustedcount'] = df_authors['adjustedcount'].astype(float)
df_authors['year'] = df_authors['year'].astype(float)
df_authors = df_authors.rename(columns={"title": "field", "name": "author"})
df_works = df_authors

df_works_author = df_works.groupby("author").sum().reset_index()
df_awards = df.assign(
    award_count=1).groupby("author-name").sum().drop(
    columns=["year"]).reset_index()

merged_df = df_awards.merge(df_works_author,
                            left_on='author-name',
                            right_on='author',
                            how='right')

merged_df = merged_df.fillna(0).drop(columns=["year"])


mod = sm.OLS(merged_df.award_count,
             sm.add_constant(merged_df[["count"]]))
res = mod.fit()
print(res.summary())


mod = sm.OLS(merged_df.award_count,
             sm.add_constant(merged_df[["adjustedcount",
                                        "count"]]))
res = mod.fit()
print(res.summary())

                            OLS Regression Results                            
Dep. Variable:            award_count   R-squared:                       0.918
Model:                            OLS   Adj. R-squared:                  0.836
Method:                 Least Squares   F-statistic:                     11.21
Date:                Fri, 13 May 2022   Prob (F-statistic):              0.185
Time:                        01:23:34   Log-Likelihood:                 1.7531
No. Observations:                   3   AIC:                            0.4938
Df Residuals:                       1   BIC:                            -1.309
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          2.2540      0.589      3.825      0.1

c:\tools\miniconda3\envs\6s079\lib\site-packages\statsmodels\stats\stattools.py:74: ValueWarning: omni_normtest is not valid with less than 8 observations; 3 samples were given.
  warn("omni_normtest is not valid with less than 8 observations; %i "
c:\tools\miniconda3\envs\6s079\lib\site-packages\statsmodels\stats\stattools.py:74: ValueWarning: omni_normtest is not valid with less than 8 observations; 3 samples were given.
  warn("omni_normtest is not valid with less than 8 observations; %i "
c:\tools\miniconda3\envs\6s079\lib\site-packages\statsmodels\regression\linear_model.py:1749: RuntimeWarning: divide by zero encountered in true_divide
  return 1 - (np.divide(self.nobs - self.k_constant, self.df_resid)
c:\tools\miniconda3\envs\6s079\lib\site-packages\statsmodels\regression\linear_model.py:1749: RuntimeWarning: invalid value encountered in double_scalars
  return 1 - (np.divide(self.nobs - self.k_constant, self.df_resid)
c:\tools\miniconda3\envs\6s079\lib\site-packages\statsmodels

# Example 2 with BrightFuture

In [4]:
df_prof = bf.load_df("profs")

# Example 2 without BrightFuture

In [5]:
df_prof = pd.read_csv('../data/profs.tsv', sep='\t', header=0)

uni_rankings = ["Massachusetts Institute of Technology",
                "Carnegie Mellon University",
                "Stanford University",
                "University of California, Berkeley",
                "University of Illinois at Urbana-Champaign",
                "Cornell University",
                "Georgia Institute of Technology",
                "University of Washington",
                "Princeton University",
                "University of Texas at Austin"]

df_prof["is_uni_top_10"] = df_prof["University"].isin(uni_rankings).astype(int)
df_prof["is_bachelors_top_10"] = df_prof["Bachelors"].isin(uni_rankings).astype(int)
df_prof["is_doctorate_top_10"] = df_prof["Doctorate"].isin(uni_rankings).astype(int)

mod = sm.OLS(df_prof.is_uni_top_10,
             sm.add_constant(df_prof[["is_bachelors_top_10",
                                      "is_doctorate_top_10"]]))
res = mod.fit()
print(res.summary())

                            OLS Regression Results                            
Dep. Variable:          is_uni_top_10   R-squared:                       0.069
Model:                            OLS   Adj. R-squared:                  0.069
Method:                 Least Squares   F-statistic:                     190.5
Date:                Fri, 13 May 2022   Prob (F-statistic):           1.59e-80
Time:                        01:23:48   Log-Likelihood:                -2424.5
No. Observations:                5144   AIC:                             4855.
Df Residuals:                    5141   BIC:                             4875.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
const                   0.1126    